In [81]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
import pyspark.sql as sql
from pyspark.sql.functions import col
import os
from pyspark.sql.functions import explode
from pyspark.sql import Window
from pyspark.sql.functions import row_number

In [3]:
conf = SparkConf().setAppName("report_analysis").setMaster('yarn')

In [4]:
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [44]:
posts_sample = spark.read.format('xml').option('rootTag', 'posts').option('rowTag', 'row').load("posts_sample.xml")
posts_sample

DataFrame[_AcceptedAnswerId: bigint, _AnswerCount: bigint, _Body: string, _ClosedDate: timestamp, _CommentCount: bigint, _CommunityOwnedDate: timestamp, _CreationDate: timestamp, _FavoriteCount: bigint, _Id: bigint, _LastActivityDate: timestamp, _LastEditDate: timestamp, _LastEditorDisplayName: string, _LastEditorUserId: bigint, _OwnerDisplayName: string, _OwnerUserId: bigint, _ParentId: bigint, _PostTypeId: bigint, _Score: bigint, _Tags: string, _Title: string, _ViewCount: bigint]

In [45]:
posts_sample.printSchema()

root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



In [77]:
posts_sample_not_null = posts_sample.select("_CreationDate", "_ViewCount", "_Tags").filter("_Tags is not null")
posts_sample_not_null.show(truncate=False)

+-----------------------+----------+------------------------------------------------------+
|_CreationDate          |_ViewCount|_Tags                                                 |
+-----------------------+----------+------------------------------------------------------+
|2008-08-01 02:42:52.667|42817     |<c#><floating-point><type-conversion><double><decimal>|
|2008-08-01 03:08:08.62 |18214     |<html><css><internet-explorer-7>                      |
|2008-08-01 04:40:59.743|555183    |<c#><.net><datetime>                                  |
|2008-08-01 04:55:37.967|149445    |<c#><datetime><time><datediff><relative-time-span>    |
|2008-08-01 05:42:38.903|176405    |<html><browser><timezone><user-agent><timezone-offset>|
|2008-08-01 05:59:11.177|123231    |<.net><math>                                          |
|2010-09-22 14:33:21.79 |3650      |<c++><character-encoding>                             |
|2010-09-23 10:47:28.92 |617       |<sharepoint><infopath>                      

In [78]:
posts_sample_not_null_only_year = posts_sample_not_null.rdd.map(lambda x: (str(x[0]).split('-')[0], x[1], x[2][1:-1].split('><'))).toDF(["year", "views", "tags"])
posts_sample_not_null_only_year = posts_sample_not_null_only_year.filter(col("year") >= 2010).filter(col("year") <= 2020)
posts_sample_not_null_only_year_explode = posts_sample_not_null_only_year.withColumn("tag", explode(posts_sample_not_null_only_year["tags"])).drop('tags')
posts_sample_not_null_only_year_explode.show(truncate=False)

+----+-----+------------------+
|year|views|tag               |
+----+-----+------------------+
|2010|3650 |c++               |
|2010|3650 |character-encoding|
|2010|617  |sharepoint        |
|2010|617  |infopath          |
|2010|1315 |iphone            |
|2010|1315 |app-store         |
|2010|1315 |in-app-purchase   |
|2010|973  |symfony1          |
|2010|973  |schema            |
|2010|973  |doctrine          |
|2010|973  |fixtures          |
|2010|132  |java              |
|2010|419  |visual-studio-2010|
|2010|419  |stylecop          |
|2010|869  |cakephp           |
|2010|869  |file-upload       |
|2010|869  |swfupload         |
|2010|1303 |git               |
|2010|1303 |cygwin            |
|2010|1303 |putty             |
+----+-----+------------------+
only showing top 20 rows



In [79]:
programming_languages = spark.read.option("header", True).option("inferSchema", True).csv("programming-languages.csv")
programming_languages.show()

+------------+--------------------+
|        name|       wikipedia_url|
+------------+--------------------+
|     A# .NET|https://en.wikipe...|
|  A# (Axiom)|https://en.wikipe...|
|  A-0 System|https://en.wikipe...|
|          A+|https://en.wikipe...|
|         A++|https://en.wikipe...|
|        ABAP|https://en.wikipe...|
|         ABC|https://en.wikipe...|
|   ABC ALGOL|https://en.wikipe...|
|       ABSET|https://en.wikipe...|
|       ABSYS|https://en.wikipe...|
|         ACC|https://en.wikipe...|
|      Accent|https://en.wikipe...|
|    Ace DASL|https://en.wikipe...|
|        ACL2|https://en.wikipe...|
|     ACT-III|https://en.wikipe...|
|     Action!|https://en.wikipe...|
|ActionScript|https://en.wikipe...|
|         Ada|https://en.wikipe...|
|     Adenine|https://en.wikipe...|
|        Agda|https://en.wikipe...|
+------------+--------------------+
only showing top 20 rows



In [82]:
names = programming_languages.select("name").rdd.map(lambda x: x[0].lower()).collect()
posts_sample_not_null_only_year_explode_lang = posts_sample_not_null_only_year_explode.filter(posts_sample_not_null_only_year_explode.tag.isin(names)).withColumnRenamed("tag", "language")
posts_sample_not_null_only_year_explode_lang.show()

+----+-----+-----------+
|year|views|   language|
+----+-----+-----------+
|2010|  132|       java|
|2010| 1258|        php|
|2010| 9649|       ruby|
|2010| 2384|          c|
|2010| 1987|        php|
|2010| 3321|     python|
|2010|  128| javascript|
|2010|  477|applescript|
|2010| 1748|        php|
|2010|  998|        php|
|2010| 2095| javascript|
|2010|  447|        sed|
|2010| 6558|     python|
|2010|  214|       java|
|2010|  214|       ruby|
|2010|  852|objective-c|
|2010|  179| javascript|
|2010| 6709|          r|
|2010|   78|        php|
|2010| 1280| javascript|
+----+-----+-----------+
only showing top 20 rows



In [83]:
posts_sample_not_null_only_year_explode_lang = posts_sample_not_null_only_year_explode_lang.groupBy("year", "language").agg({"views": "sum"})
posts_sample_not_null_only_year_explode_lang.show()

+----+-----------+----------+
|year|   language|sum(views)|
+----+-----------+----------+
|2013|     erlang|      2302|
|2017| typescript|     29031|
|2017|        sed|        93|
|2013| javascript|    609571|
|2013|         f#|      4317|
|2012| powershell|     17311|
|2019|        php|      3753|
|2017|    haskell|      4040|
|2013| autohotkey|      4421|
|2013|applescript|      2059|
|2019|      xpath|        27|
|2015|     racket|       762|
|2017|         go|      1356|
|2018|      perl6|       109|
|2015|       dart|       402|
|2015|       rust|       117|
|2012|         f#|      1222|
|2018|     python|     99996|
|2017|     prolog|        76|
|2016|      latex|       461|
+----+-----------+----------+
only showing top 20 rows



In [89]:
window = Window.partitionBy(posts_sample_not_null_only_year_explode_lang['year']).orderBy(posts_sample_not_null_only_year_explode_lang['sum(views)'].desc())
posts_sample_not_null_only_year_explode_lang_10 = posts_sample_not_null_only_year_explode_lang.select('*', row_number().over(window).alias('row_number')).filter(col('row_number') <= 10).withColumnRenamed("sum(views)", "views")
posts_sample_not_null_only_year_explode_lang_10 = posts_sample_not_null_only_year_explode_lang_10.orderBy(col("year").asc(), col("sum(views)").desc())
posts_sample_not_null_only_year_explode_lang_10.show()
posts_sample_not_null_only_year_explode_lang_10.write.mode('overwrite').parquet("top_10_programm_languages_per_year_between_2010_and_2020.parquet")

+----+-----------+-------+----------+
|year|   language|  views|row_number|
+----+-----------+-------+----------+
|2010|        php|1189629|         1|
|2010|       java| 563211|         2|
|2010| javascript| 316131|         3|
|2010|objective-c|  97009|         4|
|2010|       ruby|  76215|         5|
|2010|          c|  66587|         6|
|2010|     python|  60672|         7|
|2010|     matlab|  51865|         8|
|2010|applescript|  32305|         9|
|2010|     delphi|  13065|        10|
|2011| javascript| 809078|         1|
|2011|       java| 389834|         2|
|2011|        php| 246770|         3|
|2011|          c| 238277|         4|
|2011|objective-c| 218934|         5|
|2011|     python| 203180|         6|
|2011|       bash|  60805|         7|
|2011|       ruby|  39223|         8|
|2011|       perl|  28502|         9|
|2011|     matlab|  18816|        10|
+----+-----------+-------+----------+
only showing top 20 rows



In [90]:
posts_sample_not_null_only_year_explode_lang_10.write.mode('overwrite').parquet("top_10_languages_per_year_between_2010_and_2020.parquet")